#**Testing Dataset Capstone Project**


Setelah proses klasifikasi dilakukan untuk memprediksi cluster dari data baru (pengguna) berdasarkan pola fitur yang telah terbentuk, tahapan selanjutnya adalah melakukan pengujian model menggunakan dataset testing.

Dataset yang digunakan adalah hasil clustering `Dataset_final_input_user.csv` yang terdiri dari 20.000 data. Dari total tersebut, sebanyak 5.000 data yang sebelumnya telah dipisahkan khusus untuk tahap pengujian untuk mengevaluasi performa model akan digunakan pada tahapan ini.

# 1. Package/Library yang digunakan

Pada tahapan pertama dari proses testing akan dilakukan import package/library yang diperlukan untuk kemudahan dalam proses testing

Beberapa library utama telah dijelaskan pada notebook sebelumnya, sehingga pada bagian ini hanya dijelaskan library tambahan yang belum dibahas sebelumnya, yaitu:

*   `from tensorflow.keras.models import load_model`
    > Untuk memuat model deep learning yang sudah dilatih dan disimpan sebelumnya dalam format HDF5 atau SavedModel.

*   `from tensorflow.keras.utils import to_categorical`
    > Untuk mengubah label numerik  menjadi format one-hot encoding, yang sering digunakan dalam klasifikasi multi-kelas agar label bisa kompatibel dengan output model neural network.

*   `from sklearn.model_selection import StandardScaler`
    > Untuk menstandarisasi fitur input dengan cara mengurangi nilai rata-rata dan membaginya dengan standar deviasi, sehingga setiap fitur memiliki distribusi dengan rata-rata 0 dan varians 1.

*   `import gdown`
    > Untuk mengunduh file dari Google Drive secara langsung menggunakan URL yang dibagikan, sering dipakai untuk mengambil model atau dataset yang disimpan secara online.


In [1]:
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import gdown

# 2. Data Loading

Pada tahapan selanjutnya dilakukan proses data loading, yaitu proses memuat dataset ke dalam lingkungan kerja agar data dapat digunakan dengan lebih mudah pada tahap-tahap berikutnya.


## Mengunduh Dataset dari Google Drive

In [2]:
csv_id = '1MAeX0I6PcKJXoqTjt1-Ql4OG12lXdO5r'
csv_url = f'https://drive.google.com/uc?export=download&id={csv_id}'
csv_filename = 'nolabel_datauji_df_final_4_cluster.csv'
gdown.download(csv_url, csv_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1MAeX0I6PcKJXoqTjt1-Ql4OG12lXdO5r
To: /content/nolabel_datauji_df_final_4_cluster.csv
100%|██████████| 478k/478k [00:00<00:00, 82.8MB/s]


'nolabel_datauji_df_final_4_cluster.csv'

## Mengunduh Pipeline `preprocessing.pkl` dari Google Drive

In [3]:
scaler_id = '1bDDQrZLAkxfZc59IJHcOwU1IkgyFlO1H'
scaler_url = f'https://drive.google.com/uc?export=download&id={scaler_id}'
scaler_filename = 'model_sakoo_ann_scaler_4.pkl'
gdown.download(scaler_url, scaler_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1bDDQrZLAkxfZc59IJHcOwU1IkgyFlO1H
To: /content/model_sakoo_ann_scaler_4.pkl
100%|██████████| 4.05k/4.05k [00:00<00:00, 7.63MB/s]


'model_sakoo_ann_scaler_4.pkl'

## Mengunduh Model `model_sakoo_ann.keras` dari Google Drive

In [4]:
model_id = '1PbGDHKEQvCIIhRHnHdIyiyb4wGqG9SQ5'
model_url = f'https://drive.google.com/uc?export=download&id={model_id}'
model_filename = 'model_sakoo_ann_4.keras'
gdown.download(model_url, model_filename, quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=1PbGDHKEQvCIIhRHnHdIyiyb4wGqG9SQ5
To: /content/model_sakoo_ann_4.keras
100%|██████████| 83.5k/83.5k [00:00<00:00, 57.1MB/s]


'model_sakoo_ann_4.keras'

# 3. Data Preprocessing

Menjadikan file sebelumnya ke dalam bentuk DataFrame `df`, selanjutnyakan digunakan sebagai data baru/uji yang akan diprediksi

Kemudian membuka file scaler sebelumnya dan memuat objek tersebut kedalam variabel `scaler`


In [5]:
df = pd.read_csv(csv_filename)

X_new = df.values

with open(scaler_filename, 'rb') as f:
    scaler = pickle.load(f)
X_new_scaled = scaler.transform(X_new)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# 4. Model Loading

Memuat model deeplearning sebelumnya kedalam memori, untuk digunakan pada saat melakukan prediksi pada data

In [6]:
model = load_model(model_filename)

## Evaluasi Hasil Prediksi

Pada tahapan ini dilakukan pemeriksaan dan menambahkan hasil prediksi model ke dalam data uji sebelumnya.

In [8]:
y_pred = model.predict(X_new_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

assert len(y_pred_classes) == len(df)

df['Prediksi'] = y_pred_classes

print(df.head(10))

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
   Usia  Pendapatan Bulanan  Jumlah Tanggungan  Kebutuhan Pokok  \
0    34             4019520                  1          1169241   
1    54             4660728                  1          1663886   
2    48             3952925                  2          1352809   
3    28             4996712                  2          1966592   
4    19             5736166                  2          1472292   
5    52             5305104                  0          1573030   
6    43             5786633                  2          1541101   
7    60             1000000                  0           280359   
8    29             6644426                  3          2548638   
9    38             1178543                  3           403959   

   Tempat Tinggal  Transportasi  Pendidikan  Kesehatan  Komunikasi & Internet  \
0          517512        426117      257939     240969                 179347   
1          591577        303940      199031     246969       

**Insight**

Model berhasil memprediksi kondisi keuangan berdasarkan data testing dengan distribusi prediksi.